# 02 Make Network Version 01
Step 2 applies project cards, makes Python changes, and writes out a pickle of the version 01 network

In [27]:
import os
import sys
import yaml
from subprocess import Popen
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np


from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [30]:
input_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network')
pickle_dir = os.path.join(input_dir, 'network_standard')
output_dir = os.path.join(input_dir, 'network_standard')
card_dir = os.path.join(input_dir, 'project_cards')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')

In [31]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-08-24 20:21:52, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-08-24 20:21:52, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


### Read Version 00 Scenario

In [32]:
version_00_pickle_file_name = os.path.join(pickle_dir, 'working_scenario_00.pickle')
v_01_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

### Apply Project Cards

In [33]:
project_card_list = []
card_01_file_name = os.path.join(card_dir, 'year_2018_corrections_assign_group_roadway_class.yml')
card_02_file_name = os.path.join(card_dir, 'year_2018_corrections_assign_group.yml')

card = ProjectCard.read(card_01_file_name, validate = False)
project_card_list.append(card)

card = ProjectCard.read(card_02_file_name, validate = False)
project_card_list.append(card)

In [34]:
for project_card in project_card_list:
    v_01_scenario.apply_project(project_card)

2020-08-24 20:22:12, INFO: Applying Correct Year 2018 Assignment Group and Roadway Class
2020-08-24 20:22:12, INFO: Applying Correct Year 2018 Assignment Group and Roadway Class
2020-08-24 20:22:12, INFO: Applying Correct Year 2018 Assignment Group and Roadway Class
2020-08-24 20:22:12, INFO: Applying Correct Year 2018 Assignment Group and Roadway Class
2020-08-24 20:22:12, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group and Roadway Class
2020-08-24 20:22:12, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group and Roadway Class
2020-08-24 20:22:12, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:12, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:12, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:12, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:12, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:12, INFO: 

/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


2020-08-24 20:22:15, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:15, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:15, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:15, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group


/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


2020-08-24 20:22:18, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:18, INFO: Applying Project to Roadway Network: Correct Year 20

...

/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


2020-08-24 20:22:53, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group


/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


2020-08-24 20:22:53, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Project to Roadway Network: Correct Year 2018 Assignment Group
2020-08-24 20:22:53, INFO: Applying Project to Roadway Network: Correct Year 20

/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a co

In [35]:
v_01_scenario.applied_projects

['Correct Year 2018 Assignment Group and Roadway Class',
 'Correct Year 2018 Assignment Group']

### Make Travel Model Networks (for manual fixes)

In [36]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2020-08-24 20:23:48, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-08-24 20:23:48, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


In [37]:
m_net.roadway_standard_to_met_council_network()

2020-08-24 20:23:55, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-08-24 20:23:55, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-08-24 20:23:55, INFO: Didn't detect managed lanes in network.
2020-08-24 20:23:55, INFO: Didn't detect managed lanes in network.
2020-08-24 20:23:55, INFO: Creating calculated roadway variables.
2020-08-24 20:23:55, INFO: Creating calculated roadway variables.
2020-08-24 20:23:55, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type
2020-08-24 20:23:55, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:401: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-08-24 20:24:22, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-08-24 20:24:22, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-08-24 20:24:23, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/lasso/metcouncil_data/county/cb_2017_us_county_5m.shp
2020-08-24 20:24:23, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/lasso/metcouncil_data/county/cb_2017_us_county_5m.shp


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:293: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-08-24 20:24:55, INFO: Finished Calculating county variable: county
2020-08-24 20:24:55, INFO: Finished Calculating county variable: county
2020-08-24 20:24:56, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-08-24 20:24:56, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-08-24 20:24:56, INFO: Finished calculating centroid connector variable: centroidconnect
2020-08-24 20:24:56, INFO: Finished calculating centroid connector variable: centroidconnect
2020-08-24 20:24:56, INFO: Calculating MPO as roadway network variable: mpo
2020-08-24 20:24:56, INFO: Calculating MPO as roadway network variable: mpo
2020-08-24 20:24:56, INFO: Finished calculating MPO variable: mpo
2020-08-24 20:24:56, INFO: Finished calculating MPO variable: mpo
2020-08-24 20:24:56, INFO: Calculating Assignment Group as network variable: assign_group
2020-08-24 20:24:56, INFO: Calculating Assignment Group as network v

## Apply Manual Python Changes

#### Change 01
ROADWAY=motorway
in Areatypes 2, 3, 4, 5 -  assign_group= 1 (metered freeway)
In other areatype -  assign_group = 2 (unmetered freeway)

In [38]:
m_net.links_df.loc[(m_net.links_df['roadway'] == 'motorway') & (m_net.links_df['area_type'].isin([2,3,4,5])), 'assign_group'] = 1
m_net.links_df.loc[(m_net.links_df['roadway'] == 'motorway') & (~m_net.links_df['area_type'].isin([2,3,4,5])), 'assign_group'] = 2

#### Change 02
ROADWAY = trunk
Asgngrp=15 (expressway)

In [39]:
m_net.links_df.loc[m_net.links_df['roadway'] == 'trunk', 'assign_group'] = 15

#### Change 03
Ramps - ROADWAY_CLASS=60 is supposed to equal Ramp, but finding some issues
 
       ROADWAY_CLASS= 60 & ROADWAY='motorway'
in Areatypes 2 3 4 5 - asgngrp 3 (metered ramp)
 
In other area types -  asgngrp =4 (unmetered ramp)

In [40]:
m_net.links_df.loc[(m_net.links_df['roadway_class'] == 60) & (m_net.links_df['roadway'] == 'motorway') & (m_net.links_df['area_type'].isin([2,3,4,5])), 'assign_group'] = 3
m_net.links_df.loc[(m_net.links_df['roadway_class'] == 60) & (m_net.links_df['roadway'] == 'motorway') & (~m_net.links_df['area_type'].isin([2,3,4,5])), 'assign_group'] = 4

#### Change 04
ROADWAY_CLASS= 60 & ROADWAY='trunk'
Appears to be slip ramps and acceleration lanes on expressways. Set as assign_group=4  (might need to revisit)

In [41]:
m_net.links_df.loc[(m_net.links_df['roadway_class'] == 60) & (m_net.links_df['roadway'] == 'trunk'), 'assign_group'] = 4

#### Change 05
ROADWAY_CLASS= 60 & ROADWAY is not 'trunk' or 'motorway'
These appear to be noise and should be tagged according to the crosswalk for ROADWAY not based on their “ramp” status

In [42]:
def func(a, b, c, d):
    if a == 60:
        if b == 'trunk':
            return d
        if b == 'motorway':
            return d
        return c
    return d


df = pd.read_csv(parameters.osm_assgngrp_dict).rename(columns = {'assign_group': 'temp'})
m_net.links_df = pd.merge(m_net.links_df, df, how = 'left', on = 'roadway')
m_net.links_df['assign_group'] = np.vectorize(func)(
    m_net.links_df.roadway_class, 
    m_net.links_df.roadway, 
    m_net.links_df.temp, 
    m_net.links_df.assign_group
)
m_net.links_df = m_net.links_df.drop(columns = ['temp'])

#### Change 06
Check for any links that are coded as assign_group = 3, 4 , 13 , 14 that don't attach to asgngrp= 1, 2, or 15 – ie ramps that don’t connect to freeway or expressway
These should be flagged for review

In [43]:
join_df = pd.merge(
    m_net.links_df.loc[m_net.links_df['assign_group'].isin([3, 4, 13, 14])][['A', 'B', 'assign_group']], 
    m_net.links_df[['A', 'B', 'assign_group']].rename(columns = {'assign_group': 'next_assign_group'}),
    how = 'left', 
    left_on = 'B',  
    right_on = 'A',
).drop(columns = ['A_y', 'B_y']).rename(columns = {'A_x': 'A', 'B_x': 'B'}).dropna().astype(int)

join_df = pd.merge(
    join_df,
    m_net.links_df[['A', 'B', 'assign_group']].rename(columns = {'assign_group': 'prev_assign_group'}),
    how = 'left',
    left_on = 'A',
    right_on = 'B',
).drop(columns = ['A_y', 'B_y']).rename(columns = {'A_x': 'A', 'B_x': 'B'}).dropna().astype(int)

def func(n, p):
    if np.in1d(n, [1, 2, 15]):
        return int(1)
    elif np.in1d(p, [1, 2, 15]):
        return int (1)
    return int(0)

join_df['ramp_flag'] = np.vectorize(func)(
    join_df.next_assign_group.astype(int),
    join_df.prev_assign_group.astype(int)
)
    
join_back_df = join_df.groupby(['A','B'])['ramp_flag'].sum().reset_index()
join_back_df['ramp_flag'] = np.where(join_back_df['ramp_flag'] > 0, 'Connected', 'Need to Examine')

m_net.links_df = pd.merge(
    m_net.links_df,
    join_back_df,
    how = 'left',
    on = ['A', 'B'])


#### Change 07
Arterials – we have most of the major roads tagged as Assign_Group=6 (undivided artertial)
 
Assign_group = 6 and there is no reverse value (no B-A) then set  Assign_group = 5 (divided arterial)
If assign_group = 6 and Assign_group.R=6, then keep assign_group=6 (undivided arterial)

In [44]:
reverse_df = m_net.links_df[['A', 'B', 'drive_access']].rename(columns = {'B': 'A', 'A': 'B'})
reverse_df['reverse'] = 1
reverse_df = reverse_df[reverse_df['drive_access'] == 1].drop(columns = ['drive_access'])

join_df = pd.merge(
    m_net.links_df.loc[m_net.links_df['assign_group'] == 6][['A', 'B', 'assign_group']],
    reverse_df,
    how = 'left',
    on = ['A', 'B'],
)

join_df['update_assign_group'] = np.where(join_df['reverse'] == 1.0, 6, 5)

m_net.links_df = pd.merge(
     m_net.links_df,
     join_df.drop(columns = ['reverse', 'assign_group']),
     how = 'left',
     on = ['A', 'B'],
)

m_net.links_df.loc[(m_net.links_df['assign_group'] == 6) & (m_net.links_df['update_assign_group'] == 5), 'assign_group'] = 5
m_net.links_df = m_net.links_df.drop(columns = ['update_assign_group'])

#### Change 08
Lastly ,we need a new assign_Group value for local streets. We currently have all low level roads tagged as assign_group=7 (collector). This will give local streets too much capacity and too high of speed. I fear our assignment will be funky as the model loads too many trips onto these side streets with artificially high speeds.
 
ROADWAY=residential , set to assign_group=40 (new assigngroup for residential)  

In [46]:
# updated in lookup table, update again here for those that may be overwritten
m_net.links_df.loc[m_net.links_df['roadway'] == 'residential', 'assign_group'] = 40

## Write to Disk

In [47]:
m_net.links_metcouncil_df = m_net.links_df

### As Shapefile

In [48]:
out_cols = ['model_link_id', 'shape_id', 'assign_group', 'drive_access', 'roadway_class',
            'lanes_AM', 'lanes_MD', 'lanes_PM', 'lanes_NT',
            'roadway', 'ramp_flag', 'geometry']

m_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'v01_links.shp'),
    output_node_shp = os.path.join(output_dir, 'v01_nodes.shp'),
    link_output_variables = out_cols,
    data_to_csv = False,
    data_to_dbf = True,
)

2020-08-24 20:27:14, INFO: Writing Network as Shapefile
2020-08-24 20:27:14, INFO: Writing Network as Shapefile
2020-08-24 20:27:14, INFO: Renaming DBF Node Variables
2020-08-24 20:27:14, INFO: Renaming DBF Node Variables
2020-08-24 20:27:14, INFO: Renaming variables so that they are DBF-safe
2020-08-24 20:27:14, INFO: Renaming variables so that they are DBF-safe
2020-08-24 20:27:19, INFO: Renaming DBF Link Variables
2020-08-24 20:27:19, INFO: Renaming DBF Link Variables
2020-08-24 20:27:19, INFO: Renaming variables so that they are DBF-safe
2020-08-24 20:27:19, INFO: Renaming variables so that they are DBF-safe
2020-08-24 20:27:19, INFO: Writing Node Shapes:
 - /Users/wsp/Documents/GitHub/client_met_council_network/network_standard/v01_nodes.shp
2020-08-24 20:27:19, INFO: Writing Node Shapes:
 - /Users/wsp/Documents/GitHub/client_met_council_network/network_standard/v01_nodes.shp
2020-08-24 20:27:53, INFO: Writing Link Shapes:
 - /Users/wsp/Documents/GitHub/client_met_council_network/

### As Fixed width (for Cube)

In [23]:
m_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'cube', 'v01_links.txt'),
    output_node_txt = os.path.join(output_dir, 'cube', 'v01_nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

2020-08-18 20:24:10, INFO: Starting fixed width convertion
2020-08-18 20:24:10, INFO: Starting fixed width convertion
2020-08-18 20:26:13, INFO: Writing out link database
2020-08-18 20:26:13, INFO: Writing out link database
2020-08-18 20:26:14, INFO: Writing out link header and width ----
2020-08-18 20:26:14, INFO: Writing out link header and width ----
2020-08-18 20:26:14, INFO: Starting fixed width convertion
2020-08-18 20:26:14, INFO: Starting fixed width convertion
2020-08-18 20:26:17, INFO: Writing out node database
2020-08-18 20:26:17, INFO: Writing out node database
2020-08-18 20:26:17, INFO: Writing out node header and width
2020-08-18 20:26:17, INFO: Writing out node header and width


In [29]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_01_scenario.transit_net, parameters = parameters)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "v01_transit.lin"))

2020-08-22 20:20:13, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-08-22 20:20:13, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-08-22 20:20:13, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard
2020-08-22 20:20:13, INFO: Converting GTFS Standard Properties to MetCouncil's Cube Standard


### As Network Standard

In [33]:
karate = m_net.links_df[['model_link_id', 'assign_group']]
karate.head()

,model_link_id,assign_group
0,1,40
1,2,40
2,3,7
3,4,7
4,5,40


In [38]:
v_01_scenario.road_net.links_df = pd.merge(
            v_01_scenario.road_net.links_df,
            m_net.links_df[['model_link_id', 'assign_group']],
            how = 'left',
            on = 'model_link_id',
)

v_01_scenario.road_net.write(path = output_dir, filename = 'v01')

In [40]:
v_01_scenario.road_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'lanes', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry', 'roadway_class',
       'area_type', 'county', 'centroidconnect', 'mpo', 'assign_group'],
      dtype='object')

### Scenario as Pickle

In [41]:
scenario_filename = os.path.join(pickle_dir, 'working_scenario_01.pickle')
pickle.dump(v_01_scenario, open(scenario_filename, 'wb'))